In [26]:
# Dependencies and Setup
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt

# File to Load (Remember to Change These)
phonedata2019070930 = "201907-09phonedata.csv"
phonedata2019010630 = "201901-06phonedata.csv"
staffext = "EmployeeExtension.csv"

# Read phone Data File and store into Pandas Data Frames
phonedata20190709_df = pd.read_csv(phonedata2019070930)
phonedata20190106_df = pd.read_csv(phonedata2019010630)
staffext_df = pd.read_csv(staffext)

# Combine the data into a single dataset
phonedata2019_df = phonedata20190709_df.append(phonedata20190106_df)

# Select only Answered calls
phonedata2019_df = phonedata2019_df[phonedata2019_df.disposition == 'ANSWERED']


phonedata2019_df.tail()


calldate                          clid           src  \
30886  2019-01-02 08:37:39  "THOMAS DUNN" <+17065335571>  +17065335571   
30887  2019-01-01 21:28:41             "Amy Coutz" <119>           119   
30888  2019-01-01 21:17:48             "Amy Coutz" <119>           119   
30889  2019-01-01 09:42:32  "BRANT MYERS" <+19139634035>  +19139634035   
30890  2019-01-01 09:41:49  "BRANT MYERS" <+19139634035>  +19139634035   

          dst            dcontext                   channel  \
30886     402          ext-queues  PJSIP/Flowroute-00000ea7   
30887     *97       from-internal          SIP/119-00001d4d   
30888     *79       from-internal          SIP/119-00001d4c   
30889       s  app-announcement-3  PJSIP/Flowroute-00000ea6   
30890  vms992           ext-local  PJSIP/Flowroute-00000ea5   

                            dstchannel    lastapp  \
30886  Local/106@from-queue-00002c63;1      Queue   
30887                              NaN     Hangup   
30888                              NaN   Playback   
30889                              NaN   Playback   
30890                              NaN  VoiceMail   

                              lastdata  duration  billsec disposition  \
30886               402,trn,,,80,,,,0,       772      772    ANSWERED   
30887                              NaN       100       98    ANSWERED   
30888      do-not-disturb&de-activated         3        2    ANSWERED   
30889  custom/holiday-closure,noanswer         4        4    ANSWERED   
30890                    992@default,s        29       29    ANSWERED   

       amaflags  accountcode      uniqueid  userfield  
30886         3          NaN  1.546440e+09        NaN  
30887         3          NaN  1.546400e+09        NaN  
30888         3          NaN  1.546399e+09        NaN  
30889         3          NaN  1.546357e+09        NaN  
30890         3          NaN  1.546357e+09        NaN

In [27]:
staffext_df[['Fname','Lname']] = staffext_df.Name.str.split(" ", expand=True)
staffext_df = staffext_df.drop(['Name'], axis=1)
staffext_df.to_csv('staffext.csv', index = None, header=True)
staffext_df.tail()

Extension       Department  Fname    Lname
30        404  Client Services     CS    Sales
31        402  Client Services     CS  Support
32        403          Finance    All  Finance
33        405            Sales  Sales     East
34        401            Sales  Sales     West

In [28]:
# Set data types
phonedata2019_df['uniqueid'] = phonedata2019_df.uniqueid.astype(str)
phonedata2019_df['src'] = phonedata2019_df.src.astype(str)
phonedata2019_df['dst'] = phonedata2019_df.dst.astype(str)
phonedata2019_df['duration'] = phonedata2019_df.duration.astype(int)
phonedata2019_df = phonedata2019_df.drop(['accountcode'], axis=1)
phonedata2019_df = phonedata2019_df.drop(['amaflags'], axis=1)
phonedata2019_df = phonedata2019_df.drop(['userfield'], axis=1)
phonedata2019_df = phonedata2019_df.drop(['dcontext'], axis=1)

phonedata2019_df.head()

calldate                                 clid           src  \
0  2019-09-30 22:36:55                    "Amy Coutz" <119>           119   
1  2019-09-30 19:50:20            "TERRADATUM" <8882124793>    8882124793   
2  2019-09-30 18:53:08  "Sales:Gordy Rodney" <+18184372775>  +18184372775   
3  2019-09-30 18:46:07               "Tracy Whitmore" <107>           107   
4  2019-09-30 17:54:07     "WIRELESS CALLER" <+18016331990>  +18016331990   

           dst                   channel                       dstchannel  \
0          *78          SIP/119-00002195                              NaN   
1  13054812105          SIP/115-00002194         PJSIP/Flowroute-000010e6   
2       vmu991  PJSIP/Flowroute-000010e5                              NaN   
3          *78          SIP/107-00002193                              NaN   
4          402  PJSIP/Flowroute-000010e4  Local/106@from-queue-00006513;1   

    lastapp                                           lastdata  duration  \
0  Playback                           do-not-disturb&activated         3   
1      Dial  PJSIP/13054812105@Flowroute,300,M(setmusic^Vid...        14   
2    Hangup                                                NaN        45   
3  Playback                           do-not-disturb&activated         3   
4     Queue                                 402,trn,,,80,,,,0,       344   

   billsec disposition          uniqueid  
0        2    ANSWERED  1569901015.65569  
1        4    ANSWERED  1569891020.65565  
2       45    ANSWERED  1569887508.65489  
3        2    ANSWERED  1569887167.65488  
4      344    ANSWERED  1569884007.65433

In [29]:
# Parse columns and create dataframes for new columns
phoneuniqueid_df = phonedata2019_df.uniqueid.str.split(".", expand=True,)
phonedatetime_df = phonedata2019_df.calldate.str.split(" ", expand=True,)
phoneyyyymmdd_df = phonedatetime_df[0].str.split("-",expand=True,)

phonecallminutes = phonedata2019_df.duration
phonecallminutes_df = pd.DataFrame(phonecallminutes)
phonecallminutes_df.rename(columns = {'duration':'durationmin'}, inplace = True) 
phonecallminutes_df = phonecallminutes_df.durationmin /60
phonecallminutes_df = pd.DataFrame(phonecallminutes_df)
phonecallminutes_df = phonecallminutes_df.durationmin.round(decimals=2)
phonecallminutes_df = pd.DataFrame(phonecallminutes_df)

phoneclid_df = phonedata2019_df.clid.str.split("<", expand=True)
phoneclid_df.columns = ['clidname','clidnum']
phoneclid_df['clidname'] = phoneclid_df['clidname'].str.replace('"CID:', '', regex = True)
phoneclid_df['clidname'] = phoneclid_df['clidname'].str.replace('"DID:', '', regex = True)
phoneclid_df['clidname'] = phoneclid_df['clidname'].str.replace('"DID: ', '', regex = True)
phoneclid_df['clidname'] = phoneclid_df['clidname'].str.replace('"Sales:', '', regex = True)
phoneclid_df['clidname'] = phoneclid_df['clidname'].str.replace('"', '')
phoneclid_df['clidnum'] = phoneclid_df['clidnum'].str.replace('>', '')


In [30]:
qtr = []

for row in phoneyyyymmdd_df[1]:
    if row == '12':
        qtr.append('4')
    elif row == '11':
        qtr.append('4')
    elif row == '10':
        qtr.append('4')
    elif row == '09':
        qtr.append('3')
    elif row == '08':
        qtr.append('3')
    elif row == '07':
        qtr.append('3')
    elif row == '06':
        qtr.append('2')
    elif row == '05':
        qtr.append('2')
    elif row == '04':
        qtr.append('2')
    elif row == '03':
        qtr.append('1')
    elif row == '02':
        qtr.append('1')
    else:
        qtr.append('1')
        
phoneyyyymmdd_df['qtr'] = qtr
phoneyyyymmdd_df.head()

0   1   2 qtr
0  2019  09  30   3
1  2019  09  30   3
2  2019  09  30   3
3  2019  09  30   3
4  2019  09  30   3

In [31]:
# Add parsed columns to Phone dataframe 
phonedata2019_df["uniquecall"] = phoneuniqueid_df[0]
phonedata2019_df["subid"] = phoneuniqueid_df[1]
phonedata2019_df["qtr"] = phoneyyyymmdd_df.qtr
phonedata2019_df["year"] = phoneyyyymmdd_df[0]
phonedata2019_df["month"] = phoneyyyymmdd_df[1]
phonedata2019_df["day"] = phoneyyyymmdd_df[2]
phonedata2019_df["calldate"] = phonedatetime_df[0]
phonedata2019_df["calltime"] = phonedatetime_df[1]
phonedata2019_df["durationmin"] = phonecallminutes_df.durationmin
phonedata2019_df["clidname"] = phoneclid_df.clidname
phonedata2019_df["clidnum"] = phoneclid_df.clidnum

phonedata2019_df.sort_values(by = ['uniquecall','duration'], inplace=True)
phonedata2019_df.tail()


calldate                                 clid           src          dst  \
5  2019-09-30     "WIRELESS CALLER" <+18016331990>  +18016331990          106   
3  2019-09-30               "Tracy Whitmore" <107>           107          *78   
2  2019-09-30  "Sales:Gordy Rodney" <+18184372775>  +18184372775       vmu991   
1  2019-09-30            "TERRADATUM" <8882124793>    8882124793  13054812105   
0  2019-09-30                    "Amy Coutz" <119>           119          *78   

                           channel                dstchannel   lastapp  \
5  Local/106@from-queue-00006513;2          SIP/106-00002191      Dial   
3                 SIP/107-00002193                       NaN  Playback   
2         PJSIP/Flowroute-000010e5                       NaN    Hangup   
1                 SIP/115-00002194  PJSIP/Flowroute-000010e6      Dial   
0                 SIP/119-00002195                       NaN  Playback   

                                            lastdata  duration  billsec  \
5  SIP/99106&SIP/106&Local/90106@zulu-mobile-phon...       344      331   
3                           do-not-disturb&activated         3        2   
2                                                NaN        45       45   
1  PJSIP/13054812105@Flowroute,300,M(setmusic^Vid...        14        4   
0                           do-not-disturb&activated         3        2   

       ...       uniquecall  subid qtr  year month day  calltime durationmin  \
5      ...       1569884046  65445   3  2019    09  30  17:54:06        5.73   
3      ...       1569887167  65488   3  2019    09  30  18:46:07        0.05   
2      ...       1569887508  65489   3  2019    09  30  18:53:08        0.75   
1      ...       1569891020  65565   3  2019    09  30  19:50:20        0.23   
0      ...       1569901015  65569   3  2019    09  30  22:36:55        0.05   

           clidname       clidnum  
5  WIRELESS CALLER   +18016331990  
3   Tracy Whitmore            107  
2     Gordy Rodney   +18184372775  
1       TERRADATUM     8882124793  
0        Amy Coutz            119  

[5 rows x 22 columns]

In [32]:
# Select only unique calls
phonedata2019_df.drop_duplicates(subset = 'uniquecall', keep = 'last', inplace = True)

# Drop clid field - all data has been extracted 
phonedata2019_df = phonedata2019_df.drop(['clid'], axis=1)

# callsansweredunique_df = callsanswered_df.reindex()
# # callsansweredunique_df.head(50)
# callsansweredunique_df.uniquecall.count()
phonedata2019_df.count()

calldate       37298
src            37298
dst            37298
channel        37298
dstchannel     31650
lastapp        37298
lastdata       35855
duration       37298
billsec        37298
disposition    37298
uniqueid       37298
uniquecall     37298
subid          37298
qtr            37298
year           37298
month          37298
day            37298
calltime       37298
durationmin    37298
clidname       37298
clidnum        37298
dtype: int64

In [35]:
phonedata2019_df.to_csv('phonedata2019.csv',index = None, header=True)

In [36]:
phonedata2019_df.to_json(r'phonedata2019.json', orient="records")